Goal is to build a method that gives a better idea about relationship between target and feature. 

With categorical variables, it's simply an average by type, with the number of policies, and a chart. 

With continuous variables, it's going to be more difficult. Should bin it and then see the type of relationship (is it linear, log-linear, etc.)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_openml

# Data

Following the example here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_tweedie_regression_insurance_claims.html

I'm loading in the French Motor Dataset for now. 

In [2]:
def load_mtpl2(n_samples=None):
    """Fetch the French Motor Third-Party Liability Claims dataset.

    Parameters
    ----------
    n_samples: int, default=None
      number of samples to select (for faster run time). Full dataset has
      678013 samples.
    """
    # freMTPL2freq dataset from https://www.openml.org/d/41214
    df_freq = fetch_openml(data_id=41214, as_frame=True, parser="pandas").data
    df_freq["IDpol"] = df_freq["IDpol"].astype(int)
    df_freq.set_index("IDpol", inplace=True)

    # freMTPL2sev dataset from https://www.openml.org/d/41215
    df_sev = fetch_openml(data_id=41215, as_frame=True, parser="pandas").data

    # sum ClaimAmount over identical IDs
    df_sev = df_sev.groupby("IDpol").sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    # unquote string fields
    for column_name in df.columns[df.dtypes.values == object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]

In [20]:
df = load_mtpl2()

In [21]:
# following sklearn data manipulation steps here

# Note: filter out claims with zero amount, as the severity model
# requires strictly positive target values.
df.loc[(df["ClaimAmount"] == 0) & (df["ClaimNb"] >= 1), "ClaimNb"] = 0

# Correct for unreasonable observations (that might be data error)
# and a few exceptionally large claim amounts
df["ClaimNb"] = df["ClaimNb"].clip(upper=4)
df["Exposure"] = df["Exposure"].clip(upper=1)
# df["ClaimAmount"] = df["ClaimAmount"].clip(upper=200000)

# Categorical Variable

In [22]:
df.sample(5)

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
IDpol,,,,,,,,,,,,
1052918,0,1.00,A,6,14,37,50,B6,Regular,24,R82,0.0
2078553,0,0.90,E,9,9,66,50,B1,Regular,2740,R22,0.0
3005608,0,0.07,E,11,6,53,50,B10,Regular,2694,R54,0.0
173800,0,1.00,D,13,1,41,60,B13,Regular,993,R82,0.0
1034976,0,1.00,C,6,8,28,51,B2,Regular,145,R82,0.0


In [23]:
def get_feature_analysis_table_categorical(target, cat_feature, years_at_risk):
    
    """
    Returns a pandas DataFrame with aggregated values by feature. 
    For now, I'm thinking ClaimAmount. 
    """
    
    df = pd.DataFrame.from_dict({'target': target, 
                                 'cat_feature': cat_feature, 
                                 'years_at_risk': years_at_risk})
    
    df = df.groupby('cat_feature').agg(
        number_of_policies = ('cat_feature', 'count'),
        total_years_at_risk = ('years_at_risk', np.sum),
        average_years_at_risk = ('years_at_risk', np.mean),
        total_target = ('target', np.sum),
        average_target = ('target', np.mean), 
        max_target = ('target', np.max)
    )
    
    return df

In [43]:
format_dict = {'number_of_policies': '{:,.0f}', 
              'total_years_at_risk': '{:,.0f}', 
              'average_years_at_risk': '{:.4f}', 
              'total_target': '{:,.0f}',
              'average_target': '{:,.0f}',
              'max_target': '{:,.0f}'}

In [44]:
agg_df = get_feature_analysis_table_categorical(df.ClaimAmount, df.Area, df.Exposure)

In [45]:
agg_df.style.format(format_dict)

,number_of_policies,total_years_at_risk,average_years_at_risk,total_target,average_target,max_target
cat_feature,,,,,,
A,"103,957","61,957",0.5960,"7,739,631",74,"382,955"
B,"75,459","43,002",0.5699,"8,873,980",118,"4,075,401"
C,"191,880","104,403",0.5441,"14,612,072",76,"1,404,186"
D,"151,596","77,088",0.5085,"14,486,501",96,"1,301,173"
E,"137,167","63,786",0.4650,"13,017,426",95,"774,412"
F,"17,954","8,125",0.4526,"1,179,606",66,"114,760"
